In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
from gensim.models import Word2Vec
import logging
import json
import matplotlib.pyplot as plt
%matplotlib inline
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
train_root = './data/train_preliminary/'
test_root = './data/test/'
ad = pd.read_csv(train_root+'ad.csv')
click_log = pd.read_csv(train_root +'click_log.csv')
user = pd.read_csv(train_root+'user.csv')
Tclick_log = pd.read_csv(test_root +'click_log.csv')
Tad = pd.read_csv(test_root +'ad.csv')

In [3]:
pre_vocab = json.load(open(train_root+'vocab.json','r'))

In [34]:
Allclick_ad.product_id.unique().shape,Allclick_ad.product_category.unique().shape,Allclick_ad.advertiser_id.unique().shape,Allclick_ad.industry.unique().shape

((39057,), (18,), (57870,), (332,))

In [5]:
# click_ad = pd.merge(click_log,ad, on = 'creative_id' )
# click_ad 
# Tclick_ad = pd.merge(Tclick_log,Tad, on = 'creative_id' )
# Tclick_ad 
# Allclick_ad = click_ad.append(Tclick_ad )
# Allclick_ad
Allclick_ad = pd.read_csv(train_root+'train_new_col.csv', index_col=0)
Allclick_ad

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,unique_col
0,9,30920,567330,1,504423,30673,3,32638,319,567330.0
1,15,320815,567330,1,504423,30673,3,32638,319,567330.0
2,11,355089,567330,1,504423,30673,3,32638,319,567330.0
3,9,363442,567330,1,504423,30673,3,32638,319,567330.0
4,14,370513,567330,1,504423,30673,3,32638,319,567330.0
...,...,...,...,...,...,...,...,...,...,...
63668278,49,3131983,2033604,1,1757820,\N,5,29292,109,2044903.0
63668279,4,3131984,187695,1,168395,1634,2,14494,322,163517.0
63668280,5,3131984,322785,1,290114,2031,2,19976,238,1416237.0
63668281,16,3131984,900585,1,793805,1766,2,33097,319,3719958.0


In [4]:
Allclick_ad.sort_values(by='time',inplace=True)
Allclick_ad = Allclick_ad.reset_index()
Allclick_ad.product_id = pd.to_numeric(Allclick_ad.product_id,errors = 'coerce').fillna(50_000).map(int).map(str)
Allclick_ad.product_category = (Allclick_ad.product_category+50_000).map(str)
Allclick_ad.advertiser_id = (Allclick_ad.advertiser_id +50_020).map(str)
Allclick_ad.industry = (pd.to_numeric(Allclick_ad.industry ,errors = 'coerce').fillna(400).map(int)+120_020).map(str)
Allclick_ad

,index,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,2134125,1,3969503,146454,1,131508,50000,50018,84157,120060
1,25196907,1,2267,249047,1,223979,1927,50002,65804,120342
2,13936212,1,512898,168451,1,150988,1243,50002,79986,120342
3,13936217,1,524600,168451,2,150988,1243,50002,79986,120342
4,25196909,1,348435,249047,1,223979,1927,50002,65804,120342
...,...,...,...,...,...,...,...,...,...,...
63668278,28730723,91,3205538,3680751,1,3167517,42718,50017,60279,120420
63668279,29943347,91,3392822,4425603,1,3794825,3411,50002,77300,120268
63668280,28730719,91,3773900,3680751,1,3167517,42718,50017,60279,120420
63668281,23168403,91,498277,3877769,1,3331468,26858,50003,50043,120080


In [5]:
Allclick_ad['fusion'] = Allclick_ad['product_id'] + " " + Allclick_ad['industry'] + " " + Allclick_ad['advertiser_id'] + " " + Allclick_ad['product_category']
Allclick_ad

,index,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,fusion
0,2134125,1,3969503,146454,1,131508,50000,50018,84157,120060,50000 120060 84157 50018
1,25196907,1,2267,249047,1,223979,1927,50002,65804,120342,1927 120342 65804 50002
2,13936212,1,512898,168451,1,150988,1243,50002,79986,120342,1243 120342 79986 50002
3,13936217,1,524600,168451,2,150988,1243,50002,79986,120342,1243 120342 79986 50002
4,25196909,1,348435,249047,1,223979,1927,50002,65804,120342,1927 120342 65804 50002
...,...,...,...,...,...,...,...,...,...,...,...
63668278,28730723,91,3205538,3680751,1,3167517,42718,50017,60279,120420,42718 120420 60279 50017
63668279,29943347,91,3392822,4425603,1,3794825,3411,50002,77300,120268,3411 120268 77300 50002
63668280,28730719,91,3773900,3680751,1,3167517,42718,50017,60279,120420,42718 120420 60279 50017
63668281,23168403,91,498277,3877769,1,3331468,26858,50003,50043,120080,26858 120080 50043 50003


In [6]:
%%time
group = Allclick_ad.groupby('user_id').fusion.apply(list).reset_index()
group['fusion'] = group['fusion'].apply(lambda x:' '.join(x))
group

CPU times: user 6min 55s, sys: 32.5 s, total: 7min 28s
Wall time: 7min 28s


,user_id,fusion
0,1,50000 120346 57313 50005 136 120026 59722 5000...
1,2,87 120338 72905 50002 80 120258 60706 50002 12...
2,3,36256 120420 82994 50017 40905 120420 59897 50...
3,4,1862 120258 69471 50002 50000 120045 57996 500...
4,5,50000 120317 61902 50005 50000 120420 51012 50...
...,...,...
1899995,3999996,1261 120026 61005 50002 1404 120022 57745 5000...
1899996,3999997,50000 120026 61006 50012 1246 120101 66659 500...
1899997,3999998,50000 120047 73385 50018 2344 120187 78093 500...
1899998,3999999,50000 120342 87680 50018 1062 120258 72646 500...


In [6]:
#single feature
# click_log_app = click_log.append(Tclick_log)
Allclick_ad.sort_values(by='time',inplace=True)
Allclick_ad

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,unique_col
32216896,1,3969503,146454,1,131508,\N,18,34137,40,126365.0
25196907,1,2267,249047,1,223979,1927,2,15784,322,1033102.0
13936212,1,512898,168451,1,150988,1243,2,29966,322,125212.0
13936217,1,524600,168451,2,150988,1243,2,29966,322,125212.0
25196909,1,348435,249047,1,223979,1927,2,15784,322,1033102.0
...,...,...,...,...,...,...,...,...,...,...
58813494,91,3205538,3680751,1,3167517,42718,17,10259,\N,3680751.0
60026118,91,3392822,4425603,1,3794825,3411,2,27280,248,3425814.0
58813490,91,3773900,3680751,1,3167517,42718,17,10259,\N,3680751.0
23168403,91,498277,3877769,1,3331468,26858,3,23,60,992545.0


In [7]:
Allclick_ad['unique_col'] = Allclick_ad['unique_col'].astype(int).astype(str)
group = Allclick_ad.groupby('user_id').unique_col.apply(list).reset_index()
group['unique_col'] = group['unique_col'].apply(lambda x:' '.join(x))
group

,user_id,unique_col
0,1,762312 1037233 944483 1683941 59818 969215 194...
1,2,24333 137261 39714 609050 24333 1571699 13069 ...
2,3,283776 715748 2040913 865537 3394069 2580129 2...
3,4,2114195 2540481 2232588 1645218 1230094 180318...
4,5,3831799 350898 24333 500162 1502594 106010 168...
...,...,...
1899995,3999996,1192715 201110 4016033 417864 1393593 1698535 ...
1899996,3999997,2353377 3733221 914417 875015 678165 14993 793...
1899997,3999998,162254 1921254 54150 101550 1252593 1252593 20...
1899998,3999999,994142 95215 167175 13816 918 366567 2763077 2...


In [3]:
# group.to_csv(train_root+'corpus.csv')
group = pd.read_csv(train_root+'corpus.csv',index_col=0)
group

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,creative_id
0,1,821396 209778 877468 1683713 122032 71691 1940...
1,2,63441 155822 39714 609050 1266180 441462 13069...
2,3,661347 808612 710859 825434 593522 726940 3920...
3,4,39588 589886 574787 1892854 1962706 31070 1230...
4,5,296145 350759 24333 43235 852327 1054434 12964...
...,...,...
1899995,3999996,172004 237425 54100 417864 282643 30856 116356...
1899996,3999997,271727 390953 914417 50445 641757 14993 66299 ...
1899997,3999998,100456 464055 54150 316582 882346 882346 77124...
1899998,3999999,95215 273715 146101 117043 840 366940 148209 7...


In [8]:
sentences = group.unique_col.values
# sentences
corpus=[]
for sentence in tqdm(sentences):
    corpus.append(sentence.split())
corpus

100%|██████████| 1900000/1900000 [00:12<00:00, 146392.11it/s]


[['762312',
  '1037233',
  '944483',
  '1683941',
  '59818',
  '969215',
  '1940578',
  '90170',
  '1399633',
  '605690',
  '1503656',
  '605690',
  '1359388'],
 ['24333',
  '137261',
  '39714',
  '609050',
  '24333',
  '1571699',
  '13069',
  '1696925',
  '599224',
  '1604047',
  '71557',
  '311867',
  '1068338',
  '1024894',
  '1443125',
  '1024894',
  '2085566',
  '1696925',
  '396652',
  '157180',
  '2085566',
  '1754086',
  '137261',
  '583447',
  '2085566',
  '2360026',
  '2883959',
  '2595585',
  '1231300',
  '1429815',
  '2085566',
  '738378',
  '1489835',
  '3443599',
  '2995416',
  '3335156',
  '1416414',
  '3335156',
  '60686',
  '993248',
  '625442',
  '8340',
  '738378',
  '3967839',
  '4381615'],
 ['283776',
  '715748',
  '2040913',
  '865537',
  '3394069',
  '2580129',
  '2615402',
  '1885184',
  '1828682',
  '1817324',
  '72533',
  '1014484',
  '834309',
  '3702906',
  '75991',
  '66009',
  '2081930',
  '3021441',
  '3788534',
  '2477745',
  '3466093',
  '599224',
  '96

In [14]:
%%time
length=[]
for row in corpus:
    length.append(len(row))


# from collections import Counter 
# a= Counter(length).most_common()
a = (pd.DataFrame(length))[0].value_counts().sort_index(ascending=False)
# plt.xlim(0, 500)
# plt.ylim(0, 10000)
# plt.plot(a.index,a.values)

CPU times: user 1.32 s, sys: 62.9 ms, total: 1.38 s
Wall time: 1.38 s


In [26]:
length.sort()
length[int(190_0000*0.98)]###90% 66#95% 89#98% 125

125

In [17]:
len(length)

1900000

In [7]:
model = Word2Vec(sentences=corpus, size=50, window = 10, sg=1, min_count=1, workers=30)
# model.save(train_root+"word2vec.model")

2020-05-28 12:56:54,437 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-05-28 12:56:54,439 : INFO : collecting all words and their counts
2020-05-28 12:56:54,440 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-28 12:56:54,504 : INFO : PROGRESS: at sentence #10000, processed 335578 words, keeping 270 word types
2020-05-28 12:56:54,568 : INFO : PROGRESS: at sentence #20000, processed 673775 words, keeping 283 word types
2020-05-28 12:56:54,632 : INFO : PROGRESS: at sentence #30000, processed 1012932 words, keeping 291 word types
2020-05-28 12:56:54,696 : INFO : PROGRESS: at sentence #40000, processed 1344506 words, keeping 292 word types
2020-05-28 12:56:54,761 : INFO : PROGRESS: at sentence #50000, processed 1683247 words, keeping 298 word types
2020-05-28 12:56:54,826 : INFO : PROGRESS: at sentence #60000, processed 2018755 words, keeping 302 word types
2020-05-28 12:56:54,892 : INFO : PROGRESS: at sentence #700

2020-05-28 12:56:59,131 : INFO : PROGRESS: at sentence #720000, processed 24066390 words, keeping 325 word types
2020-05-28 12:56:59,197 : INFO : PROGRESS: at sentence #730000, processed 24404756 words, keeping 325 word types
2020-05-28 12:56:59,263 : INFO : PROGRESS: at sentence #740000, processed 24737990 words, keeping 325 word types
2020-05-28 12:56:59,328 : INFO : PROGRESS: at sentence #750000, processed 25070279 words, keeping 325 word types
2020-05-28 12:56:59,393 : INFO : PROGRESS: at sentence #760000, processed 25401959 words, keeping 325 word types
2020-05-28 12:56:59,459 : INFO : PROGRESS: at sentence #770000, processed 25737883 words, keeping 325 word types
2020-05-28 12:56:59,524 : INFO : PROGRESS: at sentence #780000, processed 26071729 words, keeping 325 word types
2020-05-28 12:56:59,589 : INFO : PROGRESS: at sentence #790000, processed 26404351 words, keeping 325 word types
2020-05-28 12:56:59,654 : INFO : PROGRESS: at sentence #800000, processed 26735949 words, keepin

2020-05-28 12:57:04,157 : INFO : PROGRESS: at sentence #1450000, processed 48536415 words, keeping 329 word types
2020-05-28 12:57:04,226 : INFO : PROGRESS: at sentence #1460000, processed 48871757 words, keeping 329 word types
2020-05-28 12:57:04,303 : INFO : PROGRESS: at sentence #1470000, processed 49206200 words, keeping 329 word types
2020-05-28 12:57:04,373 : INFO : PROGRESS: at sentence #1480000, processed 49539979 words, keeping 329 word types
2020-05-28 12:57:04,443 : INFO : PROGRESS: at sentence #1490000, processed 49878274 words, keeping 329 word types
2020-05-28 12:57:04,516 : INFO : PROGRESS: at sentence #1500000, processed 50211009 words, keeping 329 word types
2020-05-28 12:57:04,586 : INFO : PROGRESS: at sentence #1510000, processed 50543883 words, keeping 329 word types
2020-05-28 12:57:04,655 : INFO : PROGRESS: at sentence #1520000, processed 50875139 words, keeping 329 word types
2020-05-28 12:57:04,724 : INFO : PROGRESS: at sentence #1530000, processed 51208986 word

2020-05-28 12:57:27,711 : INFO : EPOCH 1 - PROGRESS: at 56.24% examples, 611884 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:57:28,713 : INFO : EPOCH 1 - PROGRESS: at 59.33% examples, 615096 words/s, in_qsize 58, out_qsize 1
2020-05-28 12:57:29,728 : INFO : EPOCH 1 - PROGRESS: at 62.14% examples, 614683 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:57:30,758 : INFO : EPOCH 1 - PROGRESS: at 65.25% examples, 616737 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:57:31,758 : INFO : EPOCH 1 - PROGRESS: at 68.04% examples, 616526 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:57:32,768 : INFO : EPOCH 1 - PROGRESS: at 70.80% examples, 615801 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:57:33,769 : INFO : EPOCH 1 - PROGRESS: at 73.70% examples, 616558 words/s, in_qsize 53, out_qsize 6
2020-05-28 12:57:34,771 : INFO : EPOCH 1 - PROGRESS: at 76.57% examples, 617185 words/s, in_qsize 58, out_qsize 1
2020-05-28 12:57:35,771 : INFO : EPOCH 1 - PROGRESS: at 79.47% examples, 617870 words/s,

2020-05-28 12:58:16,285 : INFO : EPOCH 2 - PROGRESS: at 99.05% examples, 648153 words/s, in_qsize 57, out_qsize 2
2020-05-28 12:58:16,411 : INFO : worker thread finished; awaiting finish of 29 more threads
2020-05-28 12:58:16,416 : INFO : worker thread finished; awaiting finish of 28 more threads
2020-05-28 12:58:16,421 : INFO : worker thread finished; awaiting finish of 27 more threads
2020-05-28 12:58:16,425 : INFO : worker thread finished; awaiting finish of 26 more threads
2020-05-28 12:58:16,430 : INFO : worker thread finished; awaiting finish of 25 more threads
2020-05-28 12:58:16,432 : INFO : worker thread finished; awaiting finish of 24 more threads
2020-05-28 12:58:16,434 : INFO : worker thread finished; awaiting finish of 23 more threads
2020-05-28 12:58:16,441 : INFO : worker thread finished; awaiting finish of 22 more threads
2020-05-28 12:58:16,443 : INFO : worker thread finished; awaiting finish of 21 more threads
2020-05-28 12:58:16,451 : INFO : worker thread finished; a

2020-05-28 12:58:50,380 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-28 12:58:50,384 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-28 12:58:50,384 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-28 12:58:50,386 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-28 12:58:50,392 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-28 12:58:50,397 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-28 12:58:50,400 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-28 12:58:50,403 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-28 12:58:50,403 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-28 12:58:50,404 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-28 12:58:50,406 : INFO : worker thread finished; awaiting finish of 3 more thr

2020-05-28 12:59:27,134 : INFO : EPOCH 5 - PROGRESS: at 5.12% examples, 560018 words/s, in_qsize 58, out_qsize 1
2020-05-28 12:59:28,137 : INFO : EPOCH 5 - PROGRESS: at 7.92% examples, 576440 words/s, in_qsize 56, out_qsize 3
2020-05-28 12:59:29,152 : INFO : EPOCH 5 - PROGRESS: at 10.77% examples, 585570 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:59:30,158 : INFO : EPOCH 5 - PROGRESS: at 13.69% examples, 595489 words/s, in_qsize 59, out_qsize 0
2020-05-28 12:59:31,170 : INFO : EPOCH 5 - PROGRESS: at 16.49% examples, 597346 words/s, in_qsize 57, out_qsize 2
2020-05-28 12:59:32,171 : INFO : EPOCH 5 - PROGRESS: at 19.46% examples, 604677 words/s, in_qsize 57, out_qsize 2
2020-05-28 12:59:33,173 : INFO : EPOCH 5 - PROGRESS: at 22.50% examples, 612433 words/s, in_qsize 58, out_qsize 1
2020-05-28 12:59:34,194 : INFO : EPOCH 5 - PROGRESS: at 25.64% examples, 619493 words/s, in_qsize 56, out_qsize 3
2020-05-28 12:59:35,237 : INFO : EPOCH 5 - PROGRESS: at 28.95% examples, 627143 words/s, i

In [8]:
# model.save(train_root+"test.model")
# model.wv.save_word2vec_format(train_root+"w2v_300d.txt")
# model.wv.save(train_root+'advertiser_w2v_50d.npy')
np.save(train_root+'industry_w2v_50d.npy', model.wv.vectors)
vocab = {char:i for i,char in enumerate(model.wv.index2word)}

len(vocab)

332

In [9]:
json.dump(vocab,open(train_root+'industry_vocab_50d.json','w'))
vocab

{'6': 0,
 '319': 1,
 '247': 2,
 '322': 3,
 '\\N': 4,
 '317': 5,
 '54': 6,
 '238': 7,
 '60': 8,
 '297': 9,
 '242': 10,
 '5': 11,
 '73': 12,
 '248': 13,
 '47': 14,
 '88': 15,
 '289': 16,
 '318': 17,
 '326': 18,
 '25': 19,
 '21': 20,
 '291': 21,
 '246': 22,
 '296': 23,
 '329': 24,
 '36': 25,
 '252': 26,
 '183': 27,
 '40': 28,
 '253': 29,
 '202': 30,
 '259': 31,
 '26': 32,
 '203': 33,
 '233': 34,
 '321': 35,
 '27': 36,
 '288': 37,
 '320': 38,
 '200': 39,
 '24': 40,
 '147': 41,
 '205': 42,
 '232': 43,
 '207': 44,
 '161': 45,
 '215': 46,
 '231': 47,
 '250': 48,
 '293': 49,
 '28': 50,
 '34': 51,
 '86': 52,
 '292': 53,
 '82': 54,
 '217': 55,
 '216': 56,
 '148': 57,
 '74': 58,
 '300': 59,
 '13': 60,
 '277': 61,
 '29': 62,
 '109': 63,
 '261': 64,
 '2': 65,
 '230': 66,
 '133': 67,
 '255': 68,
 '35': 69,
 '107': 70,
 '333': 71,
 '154': 72,
 '57': 73,
 '8': 74,
 '89': 75,
 '81': 76,
 '328': 77,
 '87': 78,
 '295': 79,
 '45': 80,
 '302': 81,
 '221': 82,
 '220': 83,
 '176': 84,
 '84': 85,
 '167': 86,


In [23]:
model.wv[model.wv.index2word[0]]

array([-0.18080279, -0.29769677,  0.07953159, -0.3056669 ,  0.24844131,
        0.01415971,  0.16745411,  0.09745135,  0.28049156, -0.49416095,
       -0.16758595, -0.13105442,  0.20469157, -0.51275384,  0.24622928,
        0.00431896, -0.32344285,  0.09240682, -0.02515634,  0.31357846,
       -0.33293778,  0.19879384,  0.3533855 , -0.11939967, -0.26329532,
       -0.1511083 , -0.01135168,  0.5306814 ,  0.10535237,  0.08879584,
       -0.50279367, -0.13422617, -0.13025372,  0.02226347,  0.0217515 ,
       -0.22859895, -0.01232154, -0.03084017, -0.13528973, -0.11395477,
       -0.17862733,  0.2876282 ,  0.19924235, -0.2327567 ,  0.01228735,
        0.35301444,  0.0623063 ,  0.02938176,  0.58183527, -0.24024647],
      dtype=float32)

In [9]:
from glove import Glove
from glove import Corpus
sentences=corpus
dim=100
corpus_model = Corpus()
corpus_model.fit(sentences, window=66)

glove = Glove(no_components=dim, learning_rate=0.05)
glove.fit(corpus_model.matrix, epochs=10,
          no_threads=30, verbose=True)
glove.add_dictionary(corpus_model.dictionary)


Performing 10 training epochs with 30 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [6]:
glove.word_vectors[0]

array([ 2.25897857e-02,  8.10954890e-03,  5.01121848e-04, -7.29001723e-03,
       -1.58740120e-02,  1.44625285e-02, -5.00612888e-02, -2.26090124e-02,
        5.49045233e-03, -1.64554763e-02,  3.77716412e-02,  1.03135943e-02,
        2.34905119e-03, -4.49623762e-03, -3.65952617e-02, -3.17202085e-02,
        1.26249298e-02, -1.01815395e-02, -1.26720349e-02,  3.80035003e-03,
        4.28164758e-03,  1.28527510e-02, -8.57737230e-02,  1.97874900e-02,
       -3.85452919e-02, -1.14722164e-01,  1.97289770e-02,  1.04880904e-01,
       -6.65536259e-02, -2.93574172e-02, -1.47384259e-02, -6.74268708e-03,
       -6.05780292e-02, -9.10268158e-02, -9.59741558e-03,  1.24872625e-02,
       -8.79387986e-03, -3.51264595e-02,  6.40054695e-02, -3.45349538e-02,
       -1.92439086e-02, -2.49730632e-03, -1.00142439e-02,  4.15676686e-04,
       -1.91755120e-02, -1.67348471e-02,  1.03251593e-02, -1.20230193e-02,
       -2.05383545e-02,  2.04592103e-03, -1.15373093e-01, -5.54468482e-02,
        1.82142311e-02, -

In [10]:
# type(glove.word_vectors)
json.dump(glove.dictionary,open(train_root+'unique_col_glove_vocab_100d.json','w'))
np.save(train_root+'unique_col_glove_100d.npy', glove.word_vectors)


In [9]:
len(glove.word_vectors)

39057